In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q scann

In [1]:
import numpy as np 
import time
import pandas as pd 
import os
import glob
import json
import tensorflow as tf
import tensorflow_recommenders as tfrs
import csv
import gc
import itertools
# import cudf
# import cupy as cp

2023-01-19 16:00:58.492751: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 16:00:58.721139: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2023-01-19 16:00:58.721175: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-19 16:01:00.496729: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TensorFlow Recommender version: {}'.format(tfrs.__version__))

TensorFlow version: 2.11.0
TensorFlow Recommender version: v0.7.2


In [3]:
TEST_FILE = '/kaggle/input/otto-recommender-system/test.jsonl'
ITEM_DIR = '/kaggle/input/otto-preprocessed-items-1'
SCANN_DIR = '/kaggle/input/otto-basemodel-v7/serving_model/ScaNN_Model'
MODEL_DIR = '/kaggle/input/otto-basemodel-v7/serving_model/Norm_Model'
SUBMIT_FILE = '/kaggle/working/sample_submission.csv'
OUTPUT_DIR = '/kaggle/working/'

In [16]:
def preprocess(df):
    print('   Preprocessing...')
    start = time.perf_counter()
    
    session_list, item_A_list, item_A, ts_list = [], [], [], []
    ip_session, ip_item =[],[]
    
    for index in df.index:
        session_list.append([str(df['session'][index])]*500)
        item_A_list.append([str(df['events'][index][0]['aid'])]*500)
        item_A.append(str(df['events'][index][0]['aid']))
        ts_list.append([df['events'][index][0]['ts']]*500)
        
        type_check=[]
        for event in df['events'][index]:
            if event['type']=='orders':
                ip_session.append(str(df['session'][index]) + '_' + 'orders')
                ip_item.append(str(event['aid']))
                
                ip_session.append(str(df['session'][index]) + '_' + 'carts')
                ip_item.append(str(event['aid']))
                
                ip_session.append(str(df['session'][index]) + '_' + 'clicks')
                ip_item.append(str(event['aid']))
            elif event['type']=='carts':
                ip_session.append(str(df['session'][index]) + '_' + 'carts')
                ip_item.append(str(event['aid']))
                
                ip_session.append(str(df['session'][index]) + '_' + 'clicks')
                ip_item.append(str(event['aid']))
            else:
                ip_session.append(str(df['session'][index]) + '_' + 'clicks')
                ip_item.append(str(event['aid']))
                
    input_df=pd.DataFrame({
        'session_type': ip_session,
        'labels': ip_item
        })
    
    repeat_df = pd.DataFrame({
        'session' : list(itertools.chain(*session_list)),
        'item_A' : list(itertools.chain(*item_A_list)),
        'ts' : list(itertools.chain(*ts_list))
        })
    end = time.perf_counter()
    print('   ',(end - start)* 10**6)
    return item_A, input_df, repeat_df

def retrieve_items(item_A):
    print('   Retrieval...')
    start = time.perf_counter()
    
    _, retrieved_items = scann_index(tf.constant(item_A))
    retrieved_items=tf.reshape(retrieved_items,[retrieved_items.shape[0] * retrieved_items.shape[1]])
    retrieved_items=retrieved_items.numpy()
    retrieved_items=[x.decode() for x in retrieved_items]
    end = time.perf_counter()
    print('   ',(end - start)* 10**6)
    return retrieved_items

def rate_items(rep_data_df):
    print('   Rating...') 
    start = time.perf_counter()
    
    _, _, rating_data = rat_model({
    'item_A': tf.constant(rep_data_df['item_A']),
    'item_B': tf.constant(rep_data_df['item_B']),
    'ts': tf.constant(rep_data_df['ts']) 
    })
    end = time.perf_counter()
    print('   ',(end - start)* 10**6)
    return tf.reshape(rating_data, [rating_data.shape[0]])

def postprocessing(repeat_df):
    print('   Postprocessing...')
    start = time.perf_counter()
    
    repeat_df = repeat_df.sort_values(by=['session','rating'],ascending=False)
    repeat_df.to_csv(f'zrepeat_df{chunk_num}.csv', header=False, index=False)
    
    top20_df = repeat_df.groupby('session').head(20)
    click_df = top20_df.copy() 
    cart_df = top20_df.copy()
    order_df = top20_df.copy()
       
    click_df['session_type'] = click_df['session'].astype(str) + '_clicks'
    click_df = click_df[['session_type', 'labels']]
    
    cart_df['session_type'] = cart_df['session'].astype(str) + '_carts'
    cart_df = cart_df[['session_type', 'labels']]
    
    order_df['session_type'] = order_df['session'].astype(str) + '_orders'
    order_df = order_df[['session_type', 'labels']]
    
    pred_df = pd.concat([click_df, cart_df, order_df])
    
    end = time.perf_counter()
    print('   ',(end - start)* 10**6)
    return pred_df 

In [5]:
# ret_model = tf.saved_model.load(SCANN_DIR)
rat_model = tf.saved_model.load(MODEL_DIR)

2023-01-19 16:01:14.540523: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2023-01-19 16:01:14.540575: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-19 16:01:14.540609: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bdd472ed3df2): /proc/driver/nvidia/version does not exist
2023-01-19 16:01:14.540924: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other 

In [6]:
%%time
items_dataset = tf.data.Dataset.load(ITEM_DIR)
items_dataset = items_dataset.batch(batch_size=256,num_parallel_calls=tf.data.AUTOTUNE)
itemlist_unique = np.unique(np.concatenate(list(items_dataset)))

In [7]:
%%time
scann_index = tfrs.layers.factorized_top_k.ScaNN(rat_model.item_A_model, k=500)
scann_index.index_from_dataset(
  tf.data.Dataset.zip((items_dataset, items_dataset.map(rat_model.item_B_model)))
)

CPU times: user 1min 35s, sys: 43.4 s, total: 2min 18s
Wall time: 1min 38s


In [15]:
%%time
chunk_size=100
num_lines = sum(1 for line in open(TEST_FILE))
num_chunks= int(np.ceil(num_lines / chunk_size))

print('Number of lines: ',num_lines)
print('Number of chunks: ',num_chunks)

chunks = pd.read_json(TEST_FILE, lines=True, chunksize=chunk_size)

Number of lines:  1671803
Number of chunks:  16719
CPU times: user 459 ms, sys: 146 ms, total: 605 ms
Wall time: 608 ms


In [17]:
%%time
TotalRecords = 0
TotalClicks = 0
TotalCarts = 0
TotalOrders = 0
chunk_num = 0
for chunk in chunks:
    chunk_start = time.perf_counter()
    print('Processing chunk # : ', chunk_num)
    
    #Preprocessing
    item_A, input_df, repeat_df = preprocess(chunk)
            
    #Retrieve top 50 items for each session
    repeat_df['item_B'] = retrieve_items(item_A)
    
    #Rate the items: ~1(only clicks) ~2(clicks and carts) ~3(clicks, carts, orders)
    repeat_df['rating'] = rate_items(repeat_df)
    
    #Postprocessing
    del item_A
    _ = gc.collect()
    repeat_df = repeat_df.rename(columns={"item_B": "labels"})
    pred_df = postprocessing(repeat_df)
    sub_df = pd.concat([input_df,pred_df])
    sub_df = (sub_df.groupby('session_type')['labels']
               .apply(lambda x: ' '.join(set(x.dropna())))
               .reset_index())
       
    sub_df.to_csv(f'submission_{chunk_num}.csv', header=False, index=False)
        
    del input_df, repeat_df, pred_df, sub_df 
    _ = gc.collect()
    chunk_num = chunk_num + 1
    chunk_end = time.perf_counter()
    chunk_time = (chunk_end-chunk_start) * 10**6
    print('   chunk time:', chunk_time)
    if chunk_num > 4:
        break

print('Total Records :', TotalRecords)

Processing chunk # :  0
   Preprocessing...
    51018.52799998596
   Retrieval...
    49457.98199969431
   Rating...
    135250.88699952903
   Postprocessing...
    189744.90499931562
   chunk time: 987393.2250002327
Processing chunk # :  1
   Preprocessing...
    52247.91299951903
   Retrieval...
    48839.998000403284
   Rating...
    110244.38999993436
   Postprocessing...
    185688.8379998054
   chunk time: 991088.2599997421
Processing chunk # :  2
   Preprocessing...
    43188.9149995186
   Retrieval...
    50356.10499999166
   Rating...
    124155.65199989942
   Postprocessing...
    188678.73499948473
   chunk time: 967572.6579998808
Processing chunk # :  3
   Preprocessing...
    41384.28300029773
   Retrieval...
    49950.24900017597
   Rating...
    113247.32199955179
   Postprocessing...
    199592.7899997696
   chunk time: 1004113.3960003208
Processing chunk # :  4
   Preprocessing...
    37981.492999279
   Retrieval...
    45097.04499923828
   Rating...
    111558.5690004

In [ ]:
%%time
 print('writing submission file...')
df = pd.DataFrame()
df['session_type'] = ''
df['labels'] = ''
df.to_csv(SUBMIT_FILE, mode='a', index=False)
                      
csv_files = glob.glob(f'{OUTPUT_DIR}/sub*.csv')
print(csv_files)
for file in csv_files:
    df = pd.read_csv(file)
    df.to_csv(SUBMIT_FILE, mode='a', index=False)

In [ ]:
# %%time
#  print('writing cart file...')
# df = pd.DataFrame()
# df['session_type'] = ''
# df['labels'] = ''
# df.to_csv(SUBMIT_FILE, mode='a', index=False)
                      
# csv_files = glob.glob(f'{OUTPUT_DIR}/sub*.csv')
# print(csv_files)
# for file in csv_files:
#     df = pd.read_csv(file)
#     df.to_csv(SUBMIT_FILE, mode='a', index=False)

In [ ]:
# from cudf import DataFrame
# df = DataFrame()
# df['key'] = ['A', 'A', 'A', 'B', 'B', 'B']
# df['val1'] = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3']
# df['val2'] = [1,2,3,4,5,6]
# # groups = df.groupby('key')['val1'].apply(lambda x: concat(x))

# print(df.groupby('key')['val2'].agg('max'))
# print(df['val2'].max())
# for key in groups:
#     print(key)
# #     print(value)
# # # Define a function to apply to each row in a group
# # def mult(df):
# #   df['out'] = (lambda x: ' '.join(set(x.dropna())))
# #   return df

# # result = groups.apply(mult)
# # print(result)